# Lecture 2 - Text preprocessing

## Example of preprocessing

(from [article](https://www.kdnuggets.com/2017/12/general-approach-preprocessing-text-data.html) by Matthew Mayo)

Beyond the standard Python libraries, we are also using the following:

- [NLTK](http://www.nltk.org/) - The Natural Language ToolKit is one of the best-known and most-used NLP libraries in the Python ecosystem, useful for all sorts of tasks from tokenization, to stemming, to part of speech tagging, and beyond
- [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) - BeautifulSoup is a useful library for extracting data from HTML and XML documents
- [Inflect](https://pypi.org/project/inflect/) - This is a simple library for accomplishing the natural language related tasks of generating plurals, singular nouns, ordinals, and indefinite articles, and converting numbers to words

In [ ]:
import re, string, unicodedata
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('gutenberg')
nltk.download('averaged_perceptron_tagger')
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, PorterStemmer,WordNetLemmatizer
!pip install inflect
import inflect

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
    100% |████████████████████████████████| 61kB 2.2MB/s 


We need some sample text. We'll start with something very small and artificial in order to easily see the results of what we are doing step by step.

In [ ]:
sample = """<h1>Title Goes Here</h1>

<b>Bolded Text</b>
<i>Italicized Text</i>

<img src="this should all be gone"/>

<a href="this will be gone, too">But this will still be here!</a>

I run. He ran. She is running. Will they stop running?

I talked. She was talking. They talked to them about running. Who ran to the talking runner?

[Some text we don't want to keep is in here]

¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!

something... is! wrong() with.,; this :: sentence.

I can't do this anymore. I didn't know them. Why couldn't you have dinner at the restaurant?

My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.

Don't do it.... Just don't. Billy! I know what you're doing. This is a great little house you've got here.

[This is some other unwanted text]

John: "Well, well, well."
James: "There, there. There, there."

&nbsp;&nbsp;

There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.
I have to go get 2 tutus from 2 different stores, too.

22    45   1067   445

{{Here is some stuff inside of double curly braces.}}
{Here is more stuff in single curly braces.}

[DELETE]

</body>
</html>"""

A toy dataset indeed, but make no mistake; the steps we are taking here to preprocessing this data are fully transferable.

The text data preprocessing framework:

![](https://www.kdnuggets.com/wp-content/uploads/text-preprocessing-framework-2.png)



### Noise Removal

Let's loosely define noise removal as text-specific normalization tasks which often take place prior to tokenization. Some would argue that, while the other 2 major steps of the preprocessing framework (tokenization and normalization) are basically task-independent, noise removal is much more task-specific.

Sample noise removal tasks could include:

- removing text file headers, footers
- removing HTML, XML, etc. markup and metadata
- extracting valuable data from other formats, such as JSON

As you can imagine, the boundary between noise removal and data collection and assembly, on the one hand, is a fuzzy one, while the line between noise removal and normalization is blurred on the other. Given its close relationship with specific texts and their collection and assembly, many denoising tasks, such as parsing a JSON structure, would obviously need to be implemented prior to tokenization.

In our data preprocessing pipeline, we will strip away HTML markup with the help of the BeautifulSoup library, and use regular expressions to remove open and close double brackets and anything in between them (we assume this is necessary based on our sample text).

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub(r'\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

sample = denoise_text(sample)
print(sample)

Title Goes Here
Bolded Text
Italicized Text

But this will still be here!

I run. He ran. She is running. Will they stop running?

I talked. She was talking. They talked to them about running. Who ran to the talking runner?



¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!

something... is! wrong() with.,; this :: sentence.

I can't do this anymore. I didn't know them. Why couldn't you have dinner at the restaurant?

My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.

Don't do it.... Just don't. Billy! I know what you're doing. This is a great little house you've got here.



John: "Well, well, well."
James: "There, there. There, there."

  

There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.
I have to go get 2 tutus from 2 different stores, too.

22    45   1067   445

{{Here is some stuff inside of double curly braces

### Tokenization

Tokenization is a step which splits longer strings of text into smaller pieces, or tokens. Larger chunks of text can be tokenized into sentences, sentences can be tokenized into words, etc. Further processing is generally performed after a piece of text has been appropriately tokenized. Tokenization is also referred to as text segmentation or lexical analysis. Sometimes segmentation is used to refer to the breakdown of a large chunk of text into pieces larger than words (e.g. paragraphs or sentences), while tokenization is reserved for the breakdown process which results exclusively in words.

For our task, we will tokenize our sample text into a list of words. This is done using NTLK's word_tokenize() function.



In [ ]:
words = nltk.word_tokenize(sample)
print(words)

['Title', 'Goes', 'Here', 'Bolded', 'Text', 'Italicized', 'Text', 'But', 'this', 'will', 'still', 'be', 'here', '!', 'I', 'run', '.', 'He', 'ran', '.', 'She', 'is', 'running', '.', 'Will', 'they', 'stop', 'running', '?', 'I', 'talked', '.', 'She', 'was', 'talking', '.', 'They', 'talked', 'to', 'them', 'about', 'running', '.', 'Who', 'ran', 'to', 'the', 'talking', 'runner', '?', '¡Sebastián', ',', 'Nicolás', ',', 'Alejandro', 'and', 'Jéronimo', 'are', 'going', 'to', 'the', 'store', 'tomorrow', 'morning', '!', 'something', '...', 'is', '!', 'wrong', '(', ')', 'with.', ',', ';', 'this', ':', ':', 'sentence', '.', 'I', 'ca', "n't", 'do', 'this', 'anymore', '.', 'I', 'did', "n't", 'know', 'them', '.', 'Why', 'could', "n't", 'you', 'have', 'dinner', 'at', 'the', 'restaurant', '?', 'My', 'favorite', 'movie', 'franchises', ',', 'in', 'order', ':', 'Indiana', 'Jones', ';', 'Marvel', 'Cinematic', 'Universe', ';', 'Star', 'Wars', ';', 'Back', 'to', 'the', 'Future', ';', 'Harry', 'Potter', '.', 'D

###Normalization
 
Normalization generally refers to a series of related tasks meant to put all text on a level playing field: converting all text to the same case (upper or lower), removing punctuation, converting numbers to their word equivalents, and so on. Normalization puts all words on equal footing, and allows processing to proceed uniformly.

Normalizing text can mean performing a number of tasks, but for our framework we will approach normalization in 3 distinct steps: 
- stemming, 
- lemmatization,
- everything else. 

For specifics on what these distinct steps may be, [see this post](https://www.kdnuggets.com/2017/12/general-approach-preprocessing-text-data.html).

Remember, after tokenization, we are no longer working at a text level, but now at a word level. Our normalization functions, shown below, reflect this. Function names and comments should provide the necessary insight into what each does.

In [ ]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

words = normalize(words)
print(words)

['title', 'goes', 'bolded', 'text', 'italicized', 'text', 'still', 'run', 'ran', 'running', 'stop', 'running', 'talked', 'talking', 'talked', 'running', 'ran', 'talking', 'runner', 'sebastian', 'nicolas', 'alejandro', 'jeronimo', 'going', 'store', 'tomorrow', 'morning', 'something', 'wrong', 'sentence', 'ca', 'nt', 'anymore', 'nt', 'know', 'could', 'nt', 'dinner', 'restaurant', 'favorite', 'movie', 'franchises', 'order', 'indiana', 'jones', 'marvel', 'cinematic', 'universe', 'star', 'wars', 'back', 'future', 'harry', 'potter', 'nt', 'nt', 'billy', 'know', 'great', 'little', 'house', 'got', 'john', 'well', 'well', 'well', 'james', 'lot', 'reasons', 'one hundred and one', 'reasons', 'one million', 'reasons', 'actually', 'go', 'get', 'two', 'tutus', 'two', 'different', 'stores', 'twenty-two', 'forty-five', 'one thousand and sixty-seven', 'four hundred and forty-five', 'stuff', 'inside', 'double', 'curly', 'braces', 'stuff', 'single', 'curly', 'braces']


Calling the stemming and lemming functions are done as below:

In [ ]:
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

stems, lemmas = stem_and_lemmatize(words)
print('Stemmed:\n', stems)
print('\nLemmatized:\n', lemmas)

Stemmed:
 ['titl', 'goe', 'bold', 'text', 'it', 'text', 'stil', 'run', 'ran', 'run', 'stop', 'run', 'talk', 'talk', 'talk', 'run', 'ran', 'talk', 'run', 'sebast', 'nicola', 'alejandro', 'jeronimo', 'going', 'stor', 'tomorrow', 'morn', 'someth', 'wrong', 'sent', 'ca', 'nt', 'anym', 'nt', 'know', 'could', 'nt', 'din', 'resta', 'favorit', 'movy', 'franch', 'ord', 'indian', 'jon', 'marvel', 'cinem', 'univers', 'star', 'war', 'back', 'fut', 'harry', 'pot', 'nt', 'nt', 'bil', 'know', 'gre', 'littl', 'hous', 'got', 'john', 'wel', 'wel', 'wel', 'jam', 'lot', 'reason', 'one hundred and on', 'reason', 'one million', 'reason', 'act', 'go', 'get', 'two', 'tut', 'two', 'diff', 'stor', 'twenty-two', 'forty-five', 'one thousand and sixty-seven', 'four hundred and forty-five', 'stuff', 'insid', 'doubl', 'cur', 'brac', 'stuff', 'singl', 'cur', 'brac']

Lemmatized:
 ['title', 'go', 'bolded', 'text', 'italicize', 'text', 'still', 'run', 'run', 'run', 'stop', 'run', 'talk', 'talk', 'talk', 'run', 'run', '

Depending on your NLP task or preference, one of these may be more appropriate than the other. See here for a [discussion on lemmatization vs stemming](https://blog.bitext.com/what-is-the-difference-between-stemming-and-lemmatization/).

In order to resolve ambiguous cases, lemmatization usually requires tokens to be accompanied by part-of-speech tags. For example, the word lemma for rose depends on whether it is used as a noun or a verb:

In [ ]:
lemmer = WordNetLemmatizer()
print(f"noun lemmatization: {lemmer.lemmatize('rose', 'n')}")
print(f"verb lemmatization: {lemmer.lemmatize('rose', 'v')}")

noun lemmatization: rose
verb lemmatization: rise


## Exercise 1.

In [ ]:
text_1 = """Now the Children of Ilu´vatar are Elves and Men, the Firstborn
and the Followers. And amid all the splendours of the
World, its vast halls and spaces, and its wheeling fires, Ilu´vatar
chose a place for their habitation in the Deeps of Time
and in the midst of the innumerable stars. And this habitation
might seem a little thing to those who consider only the
majesty of the Ainur, and not their terrible sharpness; as who
should take the whole field of Arda for the foundation of a
pillar and so raise it until the cone of its summit were more
bitter than a needle; or who consider only the immeasurable
vastness of the World, which still the Ainur are shaping, and
not the minute precision to which they shape all things
therein. But when the Ainur had beheld this habitation in a
vision and had seen the Children of Ilu´vatar arise therein,
then many of the most mighty among them bent all their
thought and their desire towards that place. And of these
Melkor was the chief, even as he was in the beginning the
greatest of the Ainur who took part in the Music. And he
feigned, even to himself at first, that he desired to go thither
and order all things for the good of the Children of Ilu´vatar,
controlling the turmoils of the heat and the cold that had
come to pass through him. But he desired rather to subdue
to his will both Elves and Men, envying the gifts with which
Ilu´vatar promised to endow them; and he wished himself to
have subjects and servants, and to be called Lord, and to be
a master over other wills.
But the other Ainur looked upon this habitation set within
the vast spaces of the World, which the Elves call Arda,
the Earth; and their hearts rejoiced in light, and their eyes
beholding many colours were filled with gladness; but
because of the roaring of the sea they felt a great unquiet.
And they observed the winds and the air, and the matters of
which Arda was made, of iron and stone and silver and gold
and many substances: but of all these water they most greatly
praised. And it is said by the Eldar that in water there lives
yet the echo of the Music of the Ainur more than in any
substance else that is in this Earth; and many of the Children
of Ilu´vatar hearken still unsated to the voices of the Sea, and
yet know not for what they listen.
Now to water had that Ainu whom the Elves call Ulmo
turned his thought, and of all most deeply was he instructed
by Ilu´vatar in music. But of the airs and winds Manwe¨ most
had pondered, who is the noblest of the Ainur. Of the fabric
of Earth had Aule¨ thought, to whom Ilu´vatar had given skill
and knowledge scare less than to Melkor; but the delight and
pride of Aule¨ is in the deed of making, and in the thing made,
and neither in possession nor in his own mastery; wherefore
he gives and hoards not, and is free from care, passing ever
on to some new work.
And Ilu´vatar spoke to Ulmo, and said: ‘Seest thou not how
here in this little realm in the Deeps of Time Melkor hath
made war upon thy province? He hath bethought him of
bitter cold immoderate, and yet hath not destroyed the beauty
of thy fountains, nor of thy clear pools. Behold the snow,
and the cunning work of frost! Melkor hath devised heats
and fire without restraint, and hath not dried up thy desire
nor utterly quelled the music of the sea. Behold rather the
height and glory of the clouds, and the everchanging mists;
and listen to the fall of rain upon the Earth! And in these
clouds thou art drawn nearer to Manwe¨, thy friend, whom
thou lovest.’
Then Ulmo answered: ‘Truly, Water is become now fairer
than my heart imagined, neither had my secret thought conceived
the snowflake, nor in all my music was contained the
falling of the rain. I will seek Manwe¨, that he and I may make
melodies for ever to thy delight!’ And Manwe¨ and Ulmo have
from the beginning been allied, and in all things have served
most faithfully the purpose of Ilu´vatar.
But even as Ulmo spoke, and while the Ainur were yet
gazing upon this vision, it was taken away and hidden from
their sight; and it seemed to them that in that moment they
perceived a new thing, Darkness, which they had not known
before except in thought. But they had become enamoured
of the beauty of the vision and engrossed in the unfolding
of the World which came there to being, and their minds
were filled with it; for the history was incomplete and the
circles of time not full-wrought when the vision was taken
away. And some have said that the vision ceased ere the
fulfilment of the Dominion of Men and the fading of the
Firstborn; wherefore, though the Music is over all, the Valar
have not seen as with sight the Later Ages or the ending of
the World.
Then there was unrest among the Ainur; but Ilu´vatar called
to them, and said: ‘I know the desire of your minds that what
ye have seen should verily be, not only in your thought, but
even as ye yourselves are, and yet other. Therefore I say: Ea¨!
Let these things Be! And I will send forth into the Void the
Flame Imperishable, and it shall be at the heart of the World,
and the World shall Be; and those of you that will may go
down into it.’ And suddenly the Ainur saw afar off a light,
as it were a cloud with a living heart of flame; and they knew
that this was no vision only, but that Ilu´vatar had made a
new thing: Ea¨, the World that Is."""



1. Make a vocabulary that for each word contains the number of its occurencies in above text. Store the vocabulary as a list of tuples. Sort this vocabulary by the number of occurences, from biggest to smallest.  Use word_tokenize from nltk.

2. Repeat this process, but this time also convert all words to lowercase and lemmatize all verbs.

3. Use nltk.sent_tokenizer to find the longest sentence (with respect to number of characters) in the text and return the number of words in this sentence (excluding punctuation!)

4. Read about [different tokenizers](https://www.nltk.org/api/nltk.tokenize.html) from NLTK. Give example of sentence, that would be tokenized better by TweetTokenizer().tokenize() and a sentence that would be better after word_tokenize().


Hints:
- you might need to deal with \n after each line, it can be converted to a space


In [ ]:
def voc():
  tokens = nltk.word_tokenize(text_1) 
  return sorted([(word, tokens.count(word)) for word in set(tokens)], key=lambda tup: tup[1], reverse=True)

def voc2():
  tokens = nltk.word_tokenize(text_1.lower())
  lemmer = WordNetLemmatizer()
  lemmatized = [lemmer.lemmatize(word, pos='v') for word in tokens]
  return sorted([(word, lemmatized.count(word)) for word in set(lemmatized)], key=lambda tup: tup[1], reverse=True)

def longest_sentence():
  text = re.sub(r'\n', ' ', text_1.lower())
  tokens = nltk.sent_tokenize(text)
  longest = max(tokens, key=len)
  longest = re.sub(r'[^\w^\s]', '', longest)
  return len(longest.split(" "))
  
print (longest_sentence())


82


In [ ]:
# 1
vocab_1 = voc()
print (vocab_1)
num_single_words_1 = len([word_count for word_count in vocab_1 if word_count[1] == 1])

# 2
vocab_2 = voc2()
num_single_words_2 = len([word_count for word_count in vocab_2 if word_count[1] == 1])

# # 3
num_tokens = longest_sentence()


assert len(vocab_1) == 379
assert num_single_words_1 == 234
assert len(vocab_2) == 336
assert num_single_words_2 == 191
assert num_tokens == 82

[('the', 99), ('and', 59), (',', 57), ('of', 55), ('in', 23), ('to', 23), ('.', 22), (';', 15), ('that', 15), ('And', 12), ('a', 12), ('Ilu´vatar', 12), ('not', 11), ('had', 11), ('was', 10), ('Ainur', 10), ('all', 9), ('they', 9), ('he', 8), ('World', 8), ('their', 8), ('which', 7), ('it', 7), ('this', 7), ('is', 7), ('as', 6), ('have', 6), ('thought', 6), ('for', 6), (':', 6), ('thy', 6), ('But', 6), ('vision', 6), ('but', 6), ('hath', 5), ('yet', 5), ('!', 5), ('I', 5), ('most', 5), ('Ulmo', 5), ('who', 5), ('with', 5), ('were', 5), ('only', 4), ('Melkor', 4), ('be', 4), ('many', 4), ('Elves', 4), ('said', 4), ('even', 4), ('than', 4), ('Manwe¨', 4), ('Children', 4), ('upon', 4), ('habitation', 4), ('nor', 4), ('will', 4), ('made', 4), ('things', 4), ('them', 4), ('these', 4), ('thing', 4), ('Earth', 4), ('there', 3), ('music', 3), ('its', 3), ('other', 3), ('new', 3), ('water', 3), ('Arda', 3), ('his', 3), ('from', 3), ('Men', 3), ('desire', 3), ('’', 3), ('heart', 3), ('seen', 3),

## Exercise 2.



In [ ]:
raw = nltk.corpus.gutenberg.raw("burgess-busterbrown.txt")
# print(raw[:500])
words = nltk.corpus.gutenberg.words("burgess-busterbrown.txt")
# print(words[:20])
sents = nltk.corpus.gutenberg.sents("burgess-busterbrown.txt")
print(sents[:5])

[['[', 'The', 'Adventures', 'of', 'Buster', 'Bear', 'by', 'Thornton', 'W', '.', 'Burgess', '1920', ']'], ['I'], ['BUSTER', 'BEAR', 'GOES', 'FISHING'], ['Buster', 'Bear', 'yawned', 'as', 'he', 'lay', 'on', 'his', 'comfortable', 'bed', 'of', 'leaves', 'and', 'watched', 'the', 'first', 'early', 'morning', 'sunbeams', 'creeping', 'through', 'the', 'Green', 'Forest', 'to', 'chase', 'out', 'the', 'Black', 'Shadows', '.'], ['Once', 'more', 'he', 'yawned', ',', 'and', 'slowly', 'got', 'to', 'his', 'feet', 'and', 'shook', 'himself', '.']]


Using "burgess-busterbrown.txt", do the following:

1. Count the number of sentences containing word "the" (case insensitive)
2. Compute the average **word** length in the above corpus.
3. (Stemming) Find words from above file that differ after using [Porter](http://snowball.tartarus.org/algorithms/english/stemmer.html) and [Lancaster](https://www.nltk.org/_modules/nltk/stem/lancaster.html) stemming algorithms (after lowercasing the words).
4. (Lemmatization) Perform lemmatization on above corpus. Use POS tagger (defined below) to improve the lemmatizer. Give an example (from corpus) where using POS tagger helps.

In [ ]:
#solution

def num_of_the_sentences():
  the_sents = [sentence for sentence in sents if 'the ' in ' '.join(sentence).lower()]
  return len(the_sents)

print(num_of_the_sentences())

def avg_word_len():
  return sum(len(word) for word in words) / len(words)

print (avg_word_len())

def diff_porter_lancaster():
  porter = PorterStemmer()
  lancaster = LancasterStemmer()
  
  return { word for word in words if lancaster.stem(word.lower()) != porter.stem(word.lower())}
  
print(diff_porter_lancaster())

372
3.5167431313610718
{'face', 'idea', 'safe', 'intent', 'wandered', 'Jolly', 'give', 'suspiciously', 'there', 'likes', 'dozen', 'singers', 'patience', 'silly', 'whispered', 'Master', 'Unc', 'Briar', 'chase', 'scratched', 'every', 'like', 'neither', 'power', 'sheepishly', 'terrible', 'safety', 'wondered', 'CARRIES', 'home', 'care', 'Come', 'open', 'call', 'watered', 'chased', 'XVII', 'GREAT', 'feed', 'deliberately', 'furnished', 'hateful', 'was', 'move', 'only', 'Chatterer', 'Never', 'adventures', 'wonder', 'moving', 'rejoicing', 'once', 'being', 'badly', 'fly', 'LIFE', 'agreed', 'Billy', 'need', 'blackberries', 'patient', 'free', 'makes', 'Once', 'tree', 'FINE', 'shivers', 'shiny', 'gotten', 'has', 'NEWS', 'yelling', 'shy', 'testily', 'somewhere', 'selfishness', 'knife', 'Suddenly', 'nearer', 'fishermen', 'pine', 'sudden', 'intently', 'gazing', 'finer', 'III', 'spoke', 'declared', 'surprise', 'breath', 'After', 'Happy', 'news', 'Sammy', 'ALL', 'have', 'Ever', 'XIII', 'favors', 'usual

In [ ]:
# For 4:

def pt_to_wn(pos):
    """
    Takes a Penn Treebank tag and converts it to an
    appropriate WordNet equivalent for lemmatization.

    A list of Penn Treebank tags is available at:
    https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    """

    from nltk.corpus.reader.wordnet import NOUN, VERB, ADJ, ADV

    pos = pos.lower()

    if pos.startswith('jj'):
        tag = ADJ
    elif pos == 'md':
        # Modal auxiliary verbs
        tag = VERB
    elif pos.startswith('rb'):
        tag = ADV
    elif pos.startswith('vb'):
        tag = VERB
    elif pos == 'wrb':
        # Wh-adverb (how, however, whence, whenever...)
        tag = ADV
    else:
        # default to VERB
        # This is not strictly correct, but it is good
        # enough for lemmatization.
        tag = VERB

    return tag
  
  
def nltk_pos_tagger(tokens):
    """
    Takes a list of tokens and returns a list of 
    tuples [(token, wordnet_tag), ..]
    """

    # Tag tokens with part-of-speech:
    tagged = nltk.pos_tag(tokens) 

    # Convert our Treebank-style tags to WordNet-style tags.
    tagged = [(word, pt_to_wn(tag))
                     for (word, tag) in tagged]
    return tagged
  
  
def lemmatizer(tokens):
    tagged = nltk_pos_tagger(tokens)  
    
    lemmer = WordNetLemmatizer()
    
    word_net = [lemmer.lemmatize(word.lower(), pos=tag) for (word, tag) in tagged]
    nltk_pos = [lemmer.lemmatize(token.lower()) for token in tokens]
    
    print(word_net)
    print(nltk_pos)
    #write code to lemmatize tokens using taggs from nltk_pos_tagger
    
    pass
    
lemmatizer(words)
    

['[', 'the', 'adventure', 'of', 'buster', 'bear', 'by', 'thornton', 'w', '.', 'burgess', '1920', ']', 'i', 'buster', 'bear', 'go', 'fish', 'buster', 'bear', 'yawn', 'as', 'he', 'lay', 'on', 'his', 'comfortable', 'bed', 'of', 'leave', 'and', 'watch', 'the', 'first', 'early', 'morning', 'sunbeams', 'creep', 'through', 'the', 'green', 'forest', 'to', 'chase', 'out', 'the', 'black', 'shadow', '.', 'once', 'more', 'he', 'yawn', ',', 'and', 'slowly', 'get', 'to', 'his', 'feet', 'and', 'shake', 'himself', '.', 'then', 'he', 'walk', 'over', 'to', 'a', 'big', 'pine', '-', 'tree', ',', 'stand', 'up', 'on', 'his', 'hind', 'legs', ',', 'reach', 'as', 'high', 'up', 'on', 'the', 'trunk', 'of', 'the', 'tree', 'as', 'he', 'could', ',', 'and', 'scratch', 'the', 'bark', 'with', 'his', 'great', 'claw', '.', 'after', 'that', 'he', 'yawn', 'until', 'it', 'seem', 'as', 'if', 'his', 'jaw', 'would', 'crack', ',', 'and', 'then', 'sit', 'down', 'to', 'think', 'what', 'he', 'want', 'for', 'breakfast', '.', 'whil